In [1]:
import sys

from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import time
import requests
import shutil

model = VGG16(weights='imagenet')

/Users/tir/.pyenv/versions/3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# model.summary()
# _________________________________________________________________
# Layer (type)                 Output Shape              Param #
# =================================================================
# input_1 (InputLayer)         (None, 224, 224, 3)       0
# _________________________________________________________________
# block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792
# _________________________________________________________________
# block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928
# _________________________________________________________________
# block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0
# _________________________________________________________________
# block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856
# _________________________________________________________________
# block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584
# _________________________________________________________________
# block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0
# _________________________________________________________________
# block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168
# _________________________________________________________________
# block3_conv2 (Conv2D)        (None, 56, 56, 256)       590080
# _________________________________________________________________
# block3_conv3 (Conv2D)        (None, 56, 56, 256)       590080
# _________________________________________________________________
# block3_pool (MaxPooling2D)   (None, 28, 28, 256)       0
# _________________________________________________________________
# block4_conv1 (Conv2D)        (None, 28, 28, 512)       1180160
# _________________________________________________________________
# block4_conv2 (Conv2D)        (None, 28, 28, 512)       2359808
# _________________________________________________________________
# block4_conv3 (Conv2D)        (None, 28, 28, 512)       2359808
# _________________________________________________________________
# block4_pool (MaxPooling2D)   (None, 14, 14, 512)       0
# _________________________________________________________________
# block5_conv1 (Conv2D)        (None, 14, 14, 512)       2359808
# _________________________________________________________________
# block5_conv2 (Conv2D)        (None, 14, 14, 512)       2359808
# _________________________________________________________________
# block5_conv3 (Conv2D)        (None, 14, 14, 512)       2359808
# _________________________________________________________________
# block5_pool (MaxPooling2D)   (None, 7, 7, 512)         0
# _________________________________________________________________
# flatten (Flatten)            (None, 25088)             0
# _________________________________________________________________
# fc1 (Dense)                          102764544
# _________________________________________________________________
# fc2 (Dense)                  (None, 4096)              16781312      (None, 4096)
# _________________________________________________________________
# predictions (Dense)          (None, 1000)              4097000
# =================================================================
# Total params: 138,357,544
# Trainable params: 138,357,544
# Non-trainable params: 0

In [3]:
def predictions(filename_or_url="../data/elephant.jpg", ext="jpg"):
    started = time.time()
    if filename_or_url.startswith("http"):
        response = requests.get(filename_or_url, stream=True)
        with open('img.%s' % (ext), 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response
        img_path = out_file.name
        print("downloaded %s to %s" % (filename_or_url, img_path))
    else:
        img_path = filename_or_url
    
    print("%0.4f loading and resizing image" %
          (time.time() - started), file=sys.stderr)
    img = image.load_img(img_path, target_size=(224, 224))

    print("%0.4f preprocessing input..." %
          (time.time() - started), file=sys.stderr)

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Image has been converted into (1, 224, 224, 3)

    print("%0.4f predict..." % (time.time() - started), file=sys.stderr)
    return model.predict(x)

In [4]:
def histo(p):
    return "|" * int(75*p)

In [5]:
def object_detection(filename_or_url):
    preds = predictions(filename_or_url)
    
    for (_, name, p) in decode_predictions(preds, top=5)[0]:
        print("% 25s %0.4f %s" % (name, p, histo(p)))


### Lindenau-Museum
* https://speicherwolke.uni-leipzig.de/index.php/s/IvHiDl76pCQ1ulq#//K_214%20Lindenau-Museum%20Altenburg.jpg

![]("../data/K_214.png")

In [6]:
object_detection("../data/K_214.png")

0.0000 loading and resizing image
0.1392 preprocessing input...
0.1403 predict...


                     mask 0.2797 ||||||||||||||||||||
                   maraca 0.2474 ||||||||||||||||||
                     vase 0.2135 ||||||||||||||||
                  pitcher 0.0565 ||||
                   goblet 0.0539 ||||
